In [ ]:
import seaborn
from IPython import display

seaborn.set()

import utils
import clustering_functions

In [ ]:
dataset1, dataset2, dataset3 = utils.read_dataset2()
dataset1, dataset2, dataset3 = utils.normalize_dataset2(dataset1, dataset2, dataset3)

color_list = ['firebrick','darkorange','darkgoldenrod','forestgreen','dodgerblue','blueviolet','magenta', 'black']
utils.plot_dataset2(dataset1, dataset2, dataset3, color_list)

In [ ]:
clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset1.gif')

In [ ]:
clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset2.gif')

In [ ]:
clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset3.gif')

In [ ]:
%timeit clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=False, print_output=False)

In [ ]:
clustering_functions.single_linkage(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset1.gif')

In [ ]:
clustering_functions.single_linkage(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset2.gif')

In [ ]:
clustering_functions.single_linkage(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/singlelinkage_dataset3.gif')

In [ ]:
clustering_functions.complete_linkage(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset1.gif')

In [ ]:
clustering_functions.complete_linkage(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset2.gif')

In [ ]:
clustering_functions.complete_linkage(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/completelinkage_dataset3.gif')

In [ ]:
clustering_functions.group_average(dataset1, nb_of_clusters=7, dataname="dataset1", create_anim_file=True, plot_every_iter=50, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset1.gif')

In [ ]:
clustering_functions.group_average(dataset2, nb_of_clusters=3, dataname="dataset2", create_anim_file=True, plot_every_iter=20, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset2.gif')

In [ ]:
clustering_functions.group_average(dataset3, nb_of_clusters=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/groupaverage_dataset3.gif')

In [ ]:
clustering_functions.dbscan(dataset1, min_pts=230, eps=0.795, dataname="dataset1", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/dbscan_dataset1.gif')

In [ ]:
clustering_functions.dbscan(dataset2, min_pts=30, eps=0.4, dataname="dataset2", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/dbscan_dataset2.gif')

In [ ]:
clustering_functions.dbscan(dataset3, min_pts=45, eps=0.927, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/dbscan_dataset3.gif')